<a href="https://colab.research.google.com/github/VASANTHAMALIKA-2001/General-Physician-Chatbot/blob/main/General_Physician_Chatbot.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!git clone "https://github.com/VASANTHAMALIKA-2001/General-Physician-Chatbot.git"

Cloning into 'General-Physician-Chatbot'...
remote: Enumerating objects: 8, done.
remote: Counting objects: 100% (8/8), done.
remote: Compressing objects: 100% (7/7), done.
remote: Total 8 (delta 0), reused 0 (delta 0), pack-reused 0
Unpacking objects: 100% (8/8), done.


In [2]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import os

In [3]:
training_dataset = pd.read_csv("/content/General-Physician-Chatbot/DATASET/Training.csv")
testing_dataset=pd.read_csv("/content/General-Physician-Chatbot/DATASET/Testing.csv")

In [4]:
X = training_dataset.iloc[:, 0:132].values
y = training_dataset.iloc[:, -1].values

In [5]:
dimensionality_reduction = training_dataset.groupby(training_dataset['prognosis']).max()

In [6]:
from sklearn.preprocessing import LabelEncoder
labelencoder = LabelEncoder()
y = labelencoder.fit_transform(y)
 
# Splitting the dataset into training set and test set
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.25, random_state = 0)
 
# Implementing the Decision Tree Classifier
from sklearn.tree import DecisionTreeClassifier
classifier = DecisionTreeClassifier()
classifier.fit(X_train, y_train)
 
# Saving the information of columns
cols     = training_dataset.columns
cols     = cols[:-1]
 
 
# Checking the Important features
importances = classifier.feature_importances_
indices = np.argsort(importances)[::-1]
features = cols
 
# Implementing the Visual Tree
from sklearn.tree import _tree
 
# Method to simulate the working of a Chatbot by extracting and formulating questions
def execute_bot():
 
    print("Please reply with yes/Yes or no/No for the following symptoms") 
    def print_disease(node):
        #print(node)
        node = node[0]
        #print(len(node))
        val  = node.nonzero() 
        #print(val)
        disease = labelencoder.inverse_transform(val[0])
        return disease
    def tree_to_code(tree, feature_names):
        tree_ = tree.tree_
        #print(tree_)
        feature_name = [
            feature_names[i] if i != _tree.TREE_UNDEFINED else "undefined!"
            for i in tree_.feature
        ]
        #print("def tree({}):".format(", ".join(feature_names)))
        symptoms_present = []
        def recurse(node, depth):
            indent = "  " * depth
            if tree_.feature[node] != _tree.TREE_UNDEFINED:
                name = feature_name[node]
                threshold = tree_.threshold[node]
                print(name + " ?")
                ans = input()
                ans = ans.lower()
                if ans == 'yes':
                    val = 1
                else:
                    val = 0
                if  val <= threshold:
                    recurse(tree_.children_left[node], depth + 1)
                else:
                    symptoms_present.append(name)
                    recurse(tree_.children_right[node], depth + 1)
            else:
                present_disease = print_disease(tree_.value[node])
                print( "You may have " +  present_disease )
                red_cols = dimensionality_reduction.columns 
                symptoms_given = red_cols[dimensionality_reduction.loc[present_disease].values[0].nonzero()]
                print("symptoms present  " + str(list(symptoms_present)))
                print("symptoms given "  +  str(list(symptoms_given)) )  
                confidence_level = (1.0*len(symptoms_present))/len(symptoms_given)
                print("confidence level is " + str(confidence_level))
    
        recurse(0, 1)
    
    tree_to_code(classifier,cols)
 

In [7]:
execute_bot()

Please reply with yes/Yes or no/No for the following symptoms
slurred_speech ?
no
pain_behind_the_eyes ?
no
receiving_blood_transfusion ?
yes
['You may have Hepatitis B']
symptoms present  ['receiving_blood_transfusion']
symptoms given ['itching', 'fatigue', 'lethargy', 'yellowish_skin', 'dark_urine', 'loss_of_appetite', 'abdominal_pain', 'yellow_urine', 'yellowing_of_eyes', 'malaise', 'receiving_blood_transfusion', 'receiving_unsterile_injections']
confidence level is 0.08333333333333333
